In [171]:
#from google.cloud import storage
#import pandas as pd
#from io import BytesIO
#import uuid
#import os

In [224]:
def data_prep():
    storage_client = storage.Client()
    bucket = storage_client.get_bucket("ccdr-raw")
    target = storage_client.get_bucket("ccdr-cleaned")
    blobs = bucket.list_blobs()
    data = pd.DataFrame()
    for blob in blobs:
        file = blob.download_as_string()
        data = data.append(pd.read_csv(BytesIO(file)))
    data.drop(['placeholder'], inplace=True, axis=1)
    data = data.rename(columns=data.iloc[0]).drop(data.index[0])
    data = data[['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3',
                'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4',
                'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4',
                'PAY_AMT5', 'PAY_AMT6', 'default payment next month']]
    data['SEX'].fillna(data['SEX'].mode()[0], inplace=True)
    data['SEX'].replace('', data['SEX'].mode()[0], inplace=True)
    data['EDUCATION'].fillna(4, inplace=True)
    data['EDUCATION'].replace('', 4, inplace=True)
    data['EDUCATION'].replace(0, 4, inplace=True)
    data['EDUCATION'].replace(5, 4, inplace=True)
    data['EDUCATION'].replace(6, 4, inplace=True)
    data['MARRIAGE'].fillna(3, inplace=True)
    data['MARRIAGE'].replace('', 3, inplace=True)
    data['MARRIAGE'].replace(0, 3, inplace=True)
    data['AGE'].fillna(data['AGE'].median(), inplace=True)
    data['AGE'].replace('', data['AGE'].median(), inplace=True)
    data['AGE'].replace(0, data['AGE'].median(), inplace=True)
    data.iloc[:, 6:11].fillna(data.mode().iloc[0], inplace=True)
    data.iloc[:, 6:11].replace('', data.mode().iloc[0], inplace=True)
    data['default payment next month'].fillna(data['default payment next month'].mode()[0], inplace=True)
    data['default payment next month'].replace('', data['default payment next month'].mode()[0], inplace=True)
    data = data.apply(lambda x: x.fillna(x.median()), axis=0)
    data = data.apply(lambda x: x.replace(x.median()), axis=0)
    data['ID'] = data['ID'].astype('float64')
    data['ID'] = data['ID'].astype('int64')
    data['ID'] = data['ID'].apply(lambda x: uuid.uuid4())
    return data

In [ ]:
def data_prep(event, context):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket("ccdr-raw")
    target = storage_client.get_bucket("ccdr-cleaned")
    blobs = bucket.list_blobs()
    data = pd.DataFrame()
    for blob in blobs:
        file = blob.download_as_string()
        data = data.append(pd.read_csv(BytesIO(file)))
    data.drop(['placeholder'], inplace=True, axis=1)
    data = data.rename(columns=data.iloc[0]).drop(data.index[0])
    data = data[['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3',
                'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4',
                'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4',
                'PAY_AMT5', 'PAY_AMT6', 'default payment next month']]
    data['SEX'].fillna(data['SEX'].mode()[0], inplace=True)
    data['SEX'].replace('', data['SEX'].mode()[0], inplace=True)
    data['EDUCATION'].fillna(4, inplace=True)
    data['EDUCATION'].replace('', 4, inplace=True)
    data['EDUCATION'].replace(0, 4, inplace=True)
    data['EDUCATION'].replace(5, 4, inplace=True)
    data['EDUCATION'].replace(6, 4, inplace=True)
    data['MARRIAGE'].fillna(3, inplace=True)
    data['MARRIAGE'].replace('', 3, inplace=True)
    data['MARRIAGE'].replace(0, 3, inplace=True)
    data['AGE'].fillna(data['AGE'].median(), inplace=True)
    data['AGE'].replace('', data['AGE'].median(), inplace=True)
    data['AGE'].replace(0, data['AGE'].median(), inplace=True)
    data.iloc[:, 6:11].fillna(data.mode().iloc[0], inplace=True)
    data.iloc[:, 6:11].replace('', data.mode().iloc[0], inplace=True)
    data['default payment next month'].fillna(data['default payment next month'].mode()[0], inplace=True)
    data['default payment next month'].replace('', data['default payment next month'].mode()[0], inplace=True)
    data = data.apply(lambda x: x.fillna(x.median()), axis=0)
    data = data.apply(lambda x: x.replace(x.median()), axis=0)
    data['ID'] = data['ID'].astype('float64')
    data['ID'] = data['ID'].astype('int64')
    data['ID'] = data['ID'].apply(lambda x: uuid.uuid4())
    data.to_csv('/tmp/cleaned.csv', index=False)
    blob = Blob('cleaned.csv', target)
    with open('/tmp/cleaned.csv', 'rb') as my_file:
                blob.upload_from_file(my_file)
    os.remove('/tmp/cleaned.csv')

    return